In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import re

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

block_size = 8
batch_size = 128
max_iters = 1000
learning_rate = 5e-4
eval_iters = 25
dropout = 0.1
n_embd = 256
n_layer = 4
n_head = 4

SAVE_PATH = "models/model_shakespear_words.pth"


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/Users/panavshah/Desktop/Desktop - Panav’s MacBook Pro/coding_stuff/LLM/cuda/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/panavshah/Desktop/Deskto

cpu


In [2]:
text_list = []

BOOK_PATH = 'books/shakespear/'
with open(BOOK_PATH + 'julius_caesar.txt', 'r', encoding='utf-8') as f:
          text_list.append(f.read())
with open(BOOK_PATH + 'merchant_of_venice.txt', 'r', encoding='utf-8') as f:
          text_list.append(f.read())
with open(BOOK_PATH + 'romeo_and_juliet.txt', 'r', encoding='utf-8') as f:
          text_list.append(f.read())
with open(BOOK_PATH + 'macbeth.txt', 'r', encoding='utf-8') as f:
          text_list.append(f.read())
with open(BOOK_PATH + 'the_moor_of_venice.txt', 'r', encoding='utf-8') as f:
          text_list.append(f.read())

text = ""
for text_item in text_list:
    text += text_item

text[-1000:]

'ir medicinal gum. Set you down this.\nAnd say besides, that in Aleppo once,\nWhere a malignant and a turban’d Turk\nBeat a Venetian and traduc’d the state,\nI took by the throat the circumcised dog,\nAnd smote him, thus.\n\n[_Stabs himself._]\n\nLODOVICO.\nO bloody period!\n\nGRATIANO.\nAll that’s spoke is marr’d.\n\nOTHELLO.\nI kiss’d thee ere I kill’d thee. No way but this,\nKilling myself, to die upon a kiss.\n\n[_Falling upon Desdemona._]\n\nCASSIO.\nThis did I fear, but thought he had no weapon,\nFor he was great of heart.\n\nLODOVICO.\n[_To Iago._] O Spartan dog,\nMore fell than anguish, hunger, or the sea,\nLook on the tragic loading of this bed.\nThis is thy work. The object poisons sight,\nLet it be hid. Gratiano, keep the house,\nAnd seize upon the fortunes of the Moor,\nFor they succeed on you. To you, lord governor,\nRemains the censure of this hellish villain.\nThe time, the place, the torture, O, enforce it!\nMyself will straight aboard, and to the state\nThis heavy act 

In [3]:
def remove_text_in_brackets(text):
    pattern = r'\[.*?\]'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

def replace_commas_and_periods(text):
    # Replace ', ' with ' , '
    text = re.sub(r',\s', ' , ', text)
    # Replace '. ' with ' . '
    text = re.sub(r'\.\s', ' . ', text)
    return text

def lowercase_text(text):
    return text.lower()

In [4]:
# text = remove_text_in_brackets(text)
# text[-1000:]

In [5]:
text = replace_commas_and_periods(text)
text[-1000:]

' say besides , that in Aleppo once , Where a malignant and a turban’d Turk\nBeat a Venetian and traduc’d the state , I took by the throat the circumcised dog , And smote him , thus . \n[_Stabs himself._]\n\nLODOVICO . O bloody period!\n\nGRATIANO . All that’s spoke is marr’d . \nOTHELLO . I kiss’d thee ere I kill’d thee . No way but this , Killing myself , to die upon a kiss . \n[_Falling upon Desdemona._]\n\nCASSIO . This did I fear , but thought he had no weapon , For he was great of heart . \nLODOVICO . [_To Iago._] O Spartan dog , More fell than anguish , hunger , or the sea , Look on the tragic loading of this bed . This is thy work . The object poisons sight , Let it be hid . Gratiano , keep the house , And seize upon the fortunes of the Moor , For they succeed on you . To you , lord governor , Remains the censure of this hellish villain . The time , the place , the torture , O , enforce it!\nMyself will straight aboard , and to the state\nThis heavy act with heavy heart relate 

In [6]:
strings = text.split()
unique = set(strings)
vocab_size = len(unique)
print(vocab_size)
print(list(strings)[:10])

13674
['\ufeffACT', 'I', 'SCENE', 'I', '.', 'Rome', '.', 'A', 'street', '.']


In [7]:
string_to_int = { ch: i for i, ch in enumerate(unique) }
int_to_string = { i: ch for i, ch in enumerate(unique) }
encode = lambda s: [string_to_int[c] for c in s.split()]
decode = lambda l: ''.join([int_to_string[i]+" " for i in l])

encoded_hello = encode('thou a cobbler .')
decoded_hello = decode(encoded_hello)
print(encoded_hello)
print(decoded_hello)

[11408, 12682, 12160, 10214]
thou a cobbler . 


In [8]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([ 5104,  4601,  8968,  4601, 10214,  1269, 10214,  3690,  3636, 10214,
        13163,  1061,  6491,  9469,   184, 12682,  4922,  3870, 10475, 10214,
         4719, 10214,  1822,  3559,  6491,  3339, 11344,  7072,  6491,  4405,
         3339,  3559, 10214,  1863,  8933, 12682,  1869,  7247,  6491, 10702,
         3339,  6247,  6491,  6174, 12494,  6491,  3339,  5272,  6247,  4528,
         3985, 12682,  9699,  8138,  3004,  9411,  9435,  5929, 12234,  6960,
         3561,  6491,  6329,  6023, 12087,  1499,  4658, 10214,  2302,  6491,
           78,  6491, 12682,  4704, 10214,  5036, 10214, 13403,  9119,  8943,
         6453, 13359,   184,  8943,  9715,  7247,  1809, 11408,  1908,  8943,
         3612,  3998, 13597,  6884,  6491,    78,  6491,  6329,  6023,  7761])


In [9]:
n = int(len(data)*0.8)
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs: ')
print(x)
print('targets: ')
print(y)

inputs: 
tensor([[ 8735,  4756, 10214,  ...,  5629,  1023, 10816],
        [10214,  1884,  1908,  ...,  8968,  8591, 10214],
        [10214, 11598,  6491,  ...,  8080,  6491,  5014],
        ...,
        [ 3108,  3339,  6491,  ...,   515,  5172,  6491],
        [  184,  7887, 10214,  ...,   293,  1966, 11926],
        [ 1866,  1603,  3870,  ...,  1574,  6491,  1908]])
targets: 
tensor([[ 4756, 10214,  7338,  ...,  1023, 10816,  5172],
        [ 1884,  1908,  1818,  ...,  8591, 10214, 12548],
        [11598,  6491,  8362,  ...,  6491,  5014,  6491],
        ...,
        [ 3339,  6491,  9304,  ...,  5172,  6491,  2735],
        [ 7887, 10214, 10968,  ...,  1966, 11926,  3870],
        [ 1603,  3870, 12682,  ...,  6491,  1908, 12682]])


In [10]:

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    # print(f"When input is {context}, target is {target}")

In [11]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [12]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [13]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out

In [14]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [15]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

In [17]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:]
            # get the predictions
            logits, loss = self.forward(index_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
model = model.to(device)
print(model)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(context)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

GPTLanguageModel(
  (token_embedding_table): Embedding(13674, 256)
  (position_embedding_table): Embedding(8, 256)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): Linear(in_features=256, out_features=64, bias=False)
            (query): Linear(in_features=256, out_features=64, bias=False)
            (value): Linear(in_features=256, out_features=64, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (proj): Linear(in_features=256, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=256, out_features=1024, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1024, out_features=256, bias=True)
          (3): Dropout(p=0.1, inplace=False)
        )
      )
      (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine

In [379]:
# Create a PyTorch optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for iter in tqdm(range(max_iters)):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter} | train loss: {losses['train']:.3f} | val loss: {losses['val']:.3f}")

    # Sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

  0%|                                                                                                             | 0/1000 [00:00<?, ?it/s]

step: 0 | train loss: 9.574 | val loss: 9.575


  2%|██▌                                                                                                 | 25/1000 [00:13<06:29,  2.51it/s]

step: 25 | train loss: 6.810 | val loss: 6.880


  5%|█████                                                                                               | 50/1000 [00:27<06:14,  2.54it/s]

...

step: 950 | train loss: 3.013 | val loss: 6.521


 98%|████████████████████████████████████████████████████████████████████████████████████████████████▌  | 975/1000 [09:20<00:10,  2.37it/s]

step: 975 | train loss: 2.977 | val loss: 6.584


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [09:35<00:00,  1.74it/s]

3.109978437423706


In [380]:
torch.save(model.state_dict(), SAVE_PATH)

In [18]:
model_load = GPTLanguageModel(vocab_size)
model_load.load_state_dict(torch.load(SAVE_PATH))

<All keys matched successfully>

In [20]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_text = decode(model_load.generate(context, max_new_tokens=500)[0].tolist())
formatted_text = ''
generated_text_split = generated_text.split()


for i, word in enumerate(generated_text_split):
    if word.isupper() and len(word) > 2:
        formatted_text += '\n\n' + word + '\n'
        if generated_text_split[i+1] == '.':
            generated_text_split[i+1] = ''
    else:
        formatted_text += word + ' '


formatted_text = re.sub(r' \.', '.', formatted_text)
formatted_text = re.sub(r' \,', ',', formatted_text)

with open('ai_gen_shakespear.txt', 'w') as f:
    f.write(formatted_text)
print(formatted_text)

philosopher bubbles Thou’lt world—why mourn suffer’d bloodier confusions brave feverous Jessica! all—alas Such ducats! times? seeming cool: speak; cool: money; create! pre-formed pluck fame business say’t; bubbles money; illness humble same! basest way? benefit cool: Thus corruption dares; create! kissing [_Noise cool: amiss? or humble jauncing Such illness parcels stirring? cool: Love freely; obdurate jovial Nurse; seeming bubbles Thou’lt eats appeareth We’d wounds: 

WATCH
bubbles lacks sick bubbles Such doting create! parties cool: baskets freely; Fears create! canker freely; mourn 

SAMPSON
losest stairs perforce bubbles demi-god self-charity Such touching Such corruption decree; demi-god reputation whether true Struck cool: create! cool: create! guise; stirring? Jove bubbles sycamore business citizens mood quarry tak’st is’t Chain Such Particular cool: talk’st cool: money; Soldiers; ravishing freely; bubbles same! befits tak’st ducats! sheets cool: wood fever cool: affections? del